In [13]:
!pip install text2emotion

In [14]:
#Imports 
import numpy as np # Imports numpy for handling numeric data in arrays
import pickle # Imports pickle for saving and loading small data
import os # Imports  os for file handling
import os.path # Imports os.path for directory manipulation
import h5py # Imports pickle for saving and loading large data
import codecs # Used to handle the dataset
import io # Used to read the dataset data
import re # Used to clean the text via regular expresssions
import zipfile # Package to handle zipped files, specific for the downloaded dataset
import requests # Package to request from websites, used to download the dataset
import json # Import json package for handling general dataset
import yaml # Import yaml package for handling general dataset
from tensorflow.keras import Input, Model, models # Import for keras input into models, the model itself and models for loading data 
from tensorflow.keras.activations import softmax # Import for softmax activaiton function
from tensorflow.keras.layers import Embedding, LSTM, Dense  # Import for embedding, LSTM and dense layers
from tensorflow.keras.optimizers import RMSprop # Import for RMSProp automatic optimizeer
from tensorflow.keras.preprocessing.sequence import pad_sequences # Import to pad sequences to reshape data
from tensorflow.keras.preprocessing.text import Tokenizer # Import for tokeniing inputs for use
from tensorflow.keras.utils import to_categorical, plot_model # Import for one hot encoding data
import pip # Imports pip for install
import subprocess # Imports subprcoess for calling CMD of google colab
import text2emotion as te # Imports text emotion for use

In [15]:
class ChatbotBase(): # Class to hold information to both classes, acts as an abstract
    
    #Fields
    filterRegex = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n\'0123456789' # Sets up regex characters to filter
    hiddenLayers = 232 # Defines unit amount for LSTM layers 
    batchSize = 64 # Defines the batch size, for the model 
    epochAmount = 300 # Defines the amount of epochs to use
    dataSize = 3 # Int to manage datasize of general data, note that amount should be low (2-3)

    trainingModelPath = "trainingModel/model" # Defines the path to save and load the model
    inferenceModelsPath = "inferenceModels/"
    
    dataFolderDirectory = "data/" # Defines data folder

    sDatasetUrl = 'https://github.com/Lewis-C/EHU-CIS3140-Project-Artefact/raw/main/Datasets/Knowledge%20Base%20(Modified).zip' # Defines specific data url
    sDatasetFilesPath = "Knowledge Base (Modified)/" # Defines specific data path
    sDatasetDirectory = f"{dataFolderDirectory}{sDatasetFilesPath}/" # Defines specific data direcotry

    gDatasetUrl = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json' # Defines general data url
    gDatasetFile =  f"{dataFolderDirectory}train-v2.0.json" # Defines general data file
    
    beginningOfSentence = "bos " # Defines the beginning of text marker (Bos used instead of start for example to avoid accidental classifications)
    endOfSentence = " eos" # Defines the end of text marker
    
    def PickleSave(item, fileName): # Function to save items with picckle
        with open(f'{ChatbotBase.dataFolderDirectory}{fileName}.pickle', 'wb') as f: # Opens directory
            pickle.dump(item, f, protocol=pickle.HIGHEST_PROTOCOL) # Saves item 
            
    def PickleOpen(item, fileName): # Function to open items with picckle
        with open(f'{ChatbotBase.dataFolderDirectory}{fileName}.pickle', 'rb') as f: # Opens directory
            item = pickle.load(f) # loads item
            return item # Returns item
        
    def HDF5Save(item, fileName): # Function to save items with HDF5
        with h5py.File(f'{ChatbotBase.dataFolderDirectory}{fileName}.hdf5', 'w') as f: # Opens directory
            f.create_dataset(fileName, data=item) # Saves item
            
    def HDF5Open(item, fileName): # Function to open items with HDF5
        with h5py.File(f'{ChatbotBase.dataFolderDirectory}{fileName}.hdf5', 'r') as f: # Opens directory
            item = f.get(fileName)[()] # loads item as numpy array
            return item # returns item
        
    def CleanText(text): # Function for cleaning text, removing unneccesary characters and making lower case
        
        if type(text) == dict: # Makes sure text is not a dictionary (Common error due to : use in text)
            keys = list(text) # Gets the key (note there should only be one )
            totalText = "" # Empty to hold the entire text
            for key in keys: # For each key
                value = text[key] # Finds the value associaaed 
                text = f"{key} + {value}" # Appends data back together without colon
                totalText = totalText + text # adds to total string
            text = totalText # makes text the total string
            
        text = text.lower() # Converts text input to lowercase
        
        text = re.sub(r"i'm", "i am", text) # Removes potentially conflicting punctuation, swapping with full text
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"what's", "that is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"how's", "how is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"n'", "ng", text)
        text = re.sub(r"'bout", "about", text)
        text = re.sub(r"'til", "until", text)
        text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text) # Removes conflicting characters completely
        
        return text # Return cleaned text
    
    def ProcessSpecificDataset(): # Function for processing the specific datset
        queries = [] # Declares empty list for queries
        responses = [] # Declares empty list for responses
        taggedResponses = [] # Declares empty list for tagged responses
        sDatasetFilesList = os.listdir(ChatbotBase.sDatasetDirectory)
        for file in sDatasetFilesList: # For file in the list of files for dataset (there is one for each category)
            with open(ChatbotBase.sDatasetDirectory + file, "rb") as f: # Open the file
                data = yaml.safe_load(f) # Load format specific into data
                conversations = data['conversations'] # Take each "conversation" into dialogues list
            for conversation in conversations: # For every conversation in conversations
                if len(conversation) > 2: # If the conversation has more than one line (meaning multiple potential responses)
                    queries.append(ChatbotBase.CleanText(conversation[0])) # Queries is appended the one query
                    queryResponses = '' # String for holding responses
                    for response in conversation[1:]: # For every response
                        queryResponses += " " + (ChatbotBase.CleanText(response)) # Cleans text then concatenates
                    responses.append(queryResponses) # Adds this to the responses list
                elif len(conversation) == 2: # If the conversation only has two lines 
                    queries.append(ChatbotBase.CleanText(conversation[0])) # Adds the initial line as a query
                    responses.append(ChatbotBase.CleanText(conversation[1])) # Adds the response as a response
                

        for i, response  in enumerate(responses): # For every response (gets index as well)
            if (type(response) == str): # If the response is a string (verificaiton, may not be neccesary as clean text has dictionary handling)
                taggedResponses.append(f"{ChatbotBase.beginningOfSentence}{response}{ChatbotBase.endOfSentence}") # Adds to the tagged responses with tags added
            else: # If not a string
                queries.pop(i) # Pops

        encoderInputs = queries # Defines inputs for encoder using the queries cleaned
        decoderInputs = taggedResponses # Defines inputs for decoder the 
        
        return encoderInputs, decoderInputs # Return the encoder inputs and decoder inputs

    def ProcessGeneralDataset(): # Function for processing general data
        queries = [] # Defines empty list for queries
        responses = [] # Defines empty list for responses
        taggedResponses = [] # Defines empty list for tagged response
        with open(ChatbotBase.gDatasetFile, "rb") as f: # With the dataset file ope
            data = json.load(f) # Loaded file data
            for topic in data["data"]: # For each topic in the data
                for paragraph in topic["paragraphs"][:ChatbotBase.dataSize]: # For each paragraph (note that each topic only gets three to give variety)
                    for qAndA in paragraph['qas']: # For question and answer
                        try: # Try to
                            responses.append(ChatbotBase.CleanText(qAndA["answers"][0]["text"])) # Get answer first (some questions have no answers)
                            queries.append(ChatbotBase.CleanText(qAndA['question'])) # Then get the questions, cleaning at the same time
                        except:
                            continue # If failure, skip the qanda
        for response in responses: # For every response
            taggedResponses.append(f"{ChatbotBase.beginningOfSentence}{response}{ChatbotBase.endOfSentence}") # Adds to the tagged responses with tags added
        
        encoderInputs = queries # Defines inputs for encoder using the queries cleaned
        decoderInputs = taggedResponses # Defines inputs for decoder the 
        
        return encoderInputs, decoderInputs # Return the encoder inputs and decoder inputs


In [16]:
class ChatbotTrain(ChatbotBase): # Class for training chatbot 
    wordIndex = {} # Defines empty values to load into 
    indexWord = {}
    encoderInputData = []
    decoderInputData = []
    decoderOutputData = []

    def __init__(self): # Defines the class constructor
        try: # Tries to load data from files to avoid prcessing time
            print("Attempting to find existing files")
            wordIndex = ChatbotBase.PickleOpen(self.wordIndex, "wordIndex") # Uses pickle load for word index dictionary
            indexWord = ChatbotBase.PickleOpen(self.indexWord, "indexWord") # Uses pickle load for index word dictionary
            encoderInputData = ChatbotBase.HDF5Open(self.encoderInputData, "encoderInputData")  # Uses HDF5 load for encoder input data numpy array
            decoderInputData = ChatbotBase.HDF5Open(self.decoderInputData, "decoderInputData") # Uses HDF5 load for decoder input data numpy array
            decoderOutputData = ChatbotBase.HDF5Open(self.decoderOutputData, "decoderOutputData") # Uses HDF5 load for decoder output data numpy array
            print("Existing Data Found")
        except: # If file load files
            if ((os.path.isfile(ChatbotBase.gDatasetFile)) and (os.path.isfile(ChatbotBase.sDatasetDirectory + "ai.yml"))): # If datasets files are already donwloaded
                print("Dataset found, Processing") 
            else: # Otherwise download file and store accordingly 
                print("File not found, downloading. This may take some time.")
                datasetRequest = requests.get(ChatbotBase.sDatasetUrl) # Uses url to get a HTTP request for the data and hold it as the defined r, deownloading the content
                datasetContent = zipfile.ZipFile(io.BytesIO(datasetRequest.content)) # Defines the downloaded content as zip file
                datasetContent.extractall(path=ChatbotBase.dataFolderDirectory) # Extracts the file into the script directry
                datasetRequest = requests.get(ChatbotBase.gDatasetUrl) # Uses url to get a HTTP request for the data and hold it as the defined r, deownloading the content
                datasetContent = datasetRequest.json()
                with open(ChatbotBase.gDatasetFile, 'w') as f:
                    json.dump(datasetContent, f)
                print("Dataset Downloaded, Processing")
            processedData = self.ProcessDataset() # Calls dataset process function, stores output list 
            wordIndex = processedData[0] # Gets each item from the list and stores it to specific variable 
            ChatbotBase.PickleSave(wordIndex, "wordIndex") # Saves for future use
            indexWord = processedData[1]
            ChatbotBase.PickleSave(indexWord, "indexWord")
            encoderInputData = processedData[2]
            ChatbotBase.HDF5Save(encoderInputData, "encoderInputData")
            decoderInputData = processedData[3]
            ChatbotBase.HDF5Save(decoderInputData, "decoderInputData")
            decoderOutputData = processedData[4]
            ChatbotBase.HDF5Save(decoderOutputData, "decoderOutputData")
        finally: # Regardless of route
            vocabSize = len(wordIndex) + 1 # Find the vocab size from the word index 
            encoderMaxLength = len(encoderInputData[0]) # Find the encoders max length
            decoderMaxLength = len(decoderInputData[0]) # Find the decoders max length
            ChatbotBase.PickleSave(encoderMaxLength, "encoderMaxLength") # Saves for use in web app
            ChatbotBase.PickleSave(decoderMaxLength, "decoderMaxLength")
            print("Complete")
    
        trainingModel = self.MakeChatbotModel(vocabSize) # Makes training model
        
        learningHistory = trainingModel.fit(x = [encoderInputData, decoderInputData], y = decoderOutputData, batch_size = ChatbotBase.batchSize, epochs = ChatbotBase.epochAmount, validation_split = 0.2) # Fits model to data with parameters selected and reccomended validation split of 0.2
        learningHistoryData = learningHistory.history # Stores the history of the process
        trainingModel.save(ChatbotBase.trainingModelPath) # Saves the model
        ChatbotBase.PickleSave(learningHistoryData, "history") # Saves the history

    def ProcessDataset(self):
    
        specificData = ChatbotBase.ProcessSpecificDataset() # Cals the processing of specific dataset
        encoderInputs = specificData[0] # Gets the encoder inputs
        decoderInputs = specificData[1] # Gets the decoder inputs
            
        generalData = ChatbotBase.ProcessGeneralDataset() # Calls the processing of general dataset
        encoderInputs = encoderInputs + (generalData[0]) # Adds the outputs to the encoder inputs
        decoderInputs = decoderInputs + (generalData[1]) # Adds the outputs to the decoder inputs

        print("Emotionally Analysing training data")
        encoderInputEmotionData = np.zeros(((len(encoderInputs),5))) # Creates empty list for holding emotional input data of each encoder inpout
        for encoderIndex, encoderInput in enumerate(encoderInputs): # For loop enurmerating each encoder input
            currentEmotionList = np.zeros((5)) # Creates empty array for each encoder inpout
            emotionDictionary = te.get_emotion(encoderInput) # Establishes an emotion dictionary of each input
            for emotionIndex, emotion in enumerate(emotionDictionary): # For each emotion in said dictiionary
                currentEmotionList[emotionIndex] = (emotionDictionary[emotion]) # Add to the current input list
            encoderInputEmotionData[encoderIndex] = currentEmotionList # Add current input list to total input list
        print("Emotional Analysis Complete")

        fullText = np.concatenate((encoderInputs,decoderInputs)) # Joins the two lists as full text 

        tokenizer = Tokenizer(filters=ChatbotBase.filterRegex) # Defines tokenizer with filter of incompatible regex character
        tokenizer.fit_on_texts(fullText) # Fits filter on joined text
            
        wordIndex = tokenizer.word_index # Defines a word index using tokenizer
            
        vocabSize = len(wordIndex) + 1 # Finds the vocab size

        indexWord = {} # Defines index for index word, structured with index as key instead of word
        for word, index in wordIndex.items(): # For every word and index in word index
            indexWord[index] = word # Add to reversed index
            
        encoderSequences = tokenizer.texts_to_sequences(encoderInputs) # Converts encoder inputs to sequences using tokenizer
        for i in range(len(encoderSequences)): # For every encoder input
            encoderEmotionSequence = np.append(encoderSequences[i], encoderInputEmotionData[i]) # Append the relevant emotional data
            encoderSequences[i] = encoderEmotionSequence # adds the emotion sequence to the encoder sequence

        decoderSequences = tokenizer.texts_to_sequences(decoderInputs) # Converts encoder inputs to sequences using tokenizer
            
        encoderMaxLength = max([len(sequence) for sequence in encoderSequences]) # Finds the max length of the encoder sequences
        encoderInputData = pad_sequences(encoderSequences, maxlen=encoderMaxLength, padding='post') # Pads the data according to the max, making everything the same length 
            
        decoderMaxLength = max([len(sequence) for sequence in decoderSequences]) # Finds the max length of the decoder sequences
        decoderInputData = pad_sequences(decoderSequences, maxlen=decoderMaxLength, padding='post')  # Pads the data according to the max, making everything the same length ]
            
        for i in range(len(decoderSequences)): # For each item in tokenized answers (Not encoder fit)
            decoderSequences[i] = decoderSequences[i][1:] # Remove start tag
        decoderOutputPadding = pad_sequences(decoderSequences, maxlen=decoderMaxLength, padding='post') # Pads answers
        decoderOutputData = to_categorical(decoderOutputPadding, vocabSize, dtype = 'b') # categorises data, making each sequence a list of lists where 0 represents nothing and 1 represents a wird 
            
        return wordIndex, indexWord, encoderInputData, decoderInputData, decoderOutputData

    def MakeChatbotModel(self, vocabSize): # Function to construct the chatbot model 
        # Encoder Layer Definitions
        encoderInputLayer = Input(shape=(None,), name = "Encoder_Input_Layer") # Defines input layer with flexible shape
        encoderEmbeddingLayer = Embedding(vocabSize, ChatbotBase.hiddenLayers, mask_zero=True, name = "Encoder_Embedding_Layer") # Defines embedding layer, using the amount of unique words, hidden layers defined and ignoring 0s to create embedding matrix of entire words uses
        encoderLSTMLayer = LSTM(ChatbotBase.hiddenLayers, return_state=True, dropout=0.5, name = "Encoder_LSTM_Layer") # Defines the LSTM of the encoder with use of hidden layers, and returning values for use in the decoder
        #Decoder Layer Definitions        
        decoderInputLayer = Input(shape=(None,), name = "Decoder_Input_Layer") # Defines input layer with flexible shape
        decoderEmbeddingLayer = Embedding(vocabSize, ChatbotBase.hiddenLayers, mask_zero=True, name = "Decoder_Embedding_Layer")  # Defines embedding layer, using the amount of unique words, hidden layers defined and ignoring 0s to create embedding matrix of entire words uses
        decoderLSTMLayer = LSTM(ChatbotBase.hiddenLayers, return_state=True, return_sequences=True, dropout=0.5, name = "Decoder_LSTM_Layer") # Defines the LSTM of the decoder with use of hidden layers, and returning values for use in the dense layer
        decoderDenseLayer = Dense(vocabSize, activation=softmax, name = "Decoder_Dense_Layer") # Takes the LSTM data to find the numeric weightings for each value
        
        #Encoder
        encoderInputs = encoderInputLayer # Uses input layer definition to handle encoder inputs
        encoderEmbedding = encoderEmbeddingLayer(encoderInputs) # Uses embedding layer definition to handle encoder embedding
        _, stateH, stateC = encoderLSTMLayer(encoderEmbedding) # Uses lstm layer definition to handle encoder process and returns as states
        encoderStates = [stateH, stateC] # List to hold the states of the encoder training for interpreation use and decoder use
        
        #Decoder
        decoderInputs = decoderInputLayer # Uses input layer definition to handle decoder inputs
        decoderEmbedding = decoderEmbeddingLayer(decoderInputs) # Uses input layer definition to handle decoder inputs
        decoderOutputs, _, _ = decoderLSTMLayer(decoderEmbedding, initial_state=encoderStates) # Uses lstm layer definition to handle deocer process and returns as states
        decoderOutput = decoderDenseLayer(decoderOutputs) # Uses dense layer to get numeric weightings
        
        chatbotModel = Model(inputs = [encoderInputs, decoderInputs], outputs = decoderOutput, name = "Chatbot_Model") # Defines the layers as a single model
        chatbotModel.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics = ['accuracy']) # Compiles the model with optimizer RMSProp, loss function and accuracy monitored
        
        #chatbotModel.summary(line_length=220) # Summarises model
        if not os.path.exists('figures'): # If the directory doesnt exist
            os.makedirs('figures') # Create it
        plot_model(chatbotModel, to_file="figures/model.png", show_shapes=True) # Saves the model plot to file 
        return chatbotModel # Returns for use

In [17]:
class ChatbotRun(ChatbotBase): # Class for using chatbot 
    wordIndex = {} # Defines empty values to load into 
    indexWord = {}
    encoderInputData = []
    decoderInputData = []

    def __init__(self): # Defines the class constructor
        if (not os.path.exists(ChatbotBase.trainingModelPath)): # If model file doesnt exist
            print("Training Model")
            trainingProcess = ChatbotTrain() # Instance the training model class
        try: # Tries to load data from files to avoid prcessing time
            print("Attempting to find existing files")
            wordIndex = ChatbotBase.PickleOpen(self.wordIndex, "wordIndex")  # Tries to load data from files to avoid prcessing time
            indexWord = ChatbotBase.PickleOpen(self.indexWord, "indexWord") # Tries to load data from files to avoid prcessing time
            encoderInputData = ChatbotBase.HDF5Open(self.encoderInputData, "encoderInputData")  # Uses HDF5 load for encoder input data numpy array
            decoderInputData = ChatbotBase.HDF5Open(self.decoderInputData, "decoderInputData") # Uses HDF5 load for decoder input data numpy array
            print("Existing Data Found")
        except:
            if ((os.path.isfile(ChatbotBase.gDatasetFile)) and (os.path.isfile(ChatbotBase.sDatasetDirectory + "ai.yml"))): # If datasets files are already donwloaded
                print("Dataset found, Processing") 
            else: # Otherwise download file and store accordingly 
                print("File not found, downloading. This may take some time.")
                datasetRequest = requests.get(ChatbotBase.sDatasetUrl) # Uses url to get a HTTP request for the data and hold it as the defined r, deownloading the content
                datasetContent = zipfile.ZipFile(io.BytesIO(datasetRequest.content)) # Defines the downloaded content as zip file
                datasetContent.extractall(path=ChatbotBase.dataFolderDirectory) # Extracts the file into the script directry
                datasetRequest = requests.get(ChatbotBase.gDatasetUrl) # Uses url to get a HTTP request for the data and hold it as the defined r, deownloading the content
                datasetContent = datasetRequest.json()
                with open(ChatbotBase.gDatasetFile, 'w') as f:
                    json.dump(datasetContent, f)
                print("Dataset Downloaded, Processing")
            wordIndex, indexWord, encoderInputData, decoderInputData  = self.ProcessDataset() # Calls dataset process function, stores output into values
            ChatbotBase.PickleSave(wordIndex, "wordIndex") # Saves for future use
            ChatbotBase.PickleSave(indexWord, "indexWord")
            ChatbotBase.HDF5Save(encoderInputData, "encoderInputData")
            ChatbotBase.HDF5Save(decoderInputData, "decoderInputData")
        finally:
            vocabSize = len(wordIndex) + 1 # Find the vocab size from the word index 
            encoderMaxLength = len(encoderInputData[0])  # Find the encoders max length
            decoderMaxLength = len(decoderInputData[0])  # Find the decoders max length
            ChatbotBase.PickleSave(encoderMaxLength, "encoderMaxLength") # Saves for use in web app
            ChatbotBase.PickleSave(decoderMaxLength, "decoderMaxLength")

            print("Complete")

        self.MakeInferenceModels() # Makes inference models

    def ProcessDataset(self):
    
        specificData = ChatbotBase.ProcessSpecificDataset() # Cals the processing of specific dataset
        encoderInputs = specificData[0] # Gets the encoder inputs
        decoderInputs = specificData[1] # Gets the decoder inputs
            
        generalData = ChatbotBase.ProcessGeneralDataset() # Calls the processing of general dataset
        encoderInputs = encoderInputs + (generalData[0]) # Adds the outputs to the encoder inputs
        decoderInputs = decoderInputs + (generalData[1]) # Adds the outputs to the decoder inputs
            
        fullText = np.concatenate((encoderInputs,decoderInputs)) # Joins the two lists as full text 

        tokenizer = Tokenizer(filters=ChatbotBase.filterRegex) # Defines tokenizer with filter of incompatible regex character
        tokenizer.fit_on_texts(fullText) # Fits filter on joined text
            
        wordIndex = tokenizer.word_index # Defines a word index using tokenizer
            
        vocabSize = len(wordIndex) + 1 # Finds the vocab size

        indexWord = {} # Defines index for index word, structured with index as key instead of word
        for word, index in wordIndex.items(): # For every word and index in word index
            indexWord[index] = word # Add to reversed index
            
        encoderSequences = tokenizer.texts_to_sequences(encoderInputs) # Converts encoder inputs to sequences using tokenizer
        decoderSequences = tokenizer.texts_to_sequences(decoderInputs) # Converts encoder inputs to sequences using tokenizer
            
        encoderMaxLength = max([len(sequence) for sequence in encoderSequences]) # Finds the max length of the encoder sequences
        encoderInputData = pad_sequences(encoderSequences, maxlen=encoderMaxLength, padding='post') # Pads the data according to the max, making everything the same length 
            
        decoderMaxLength = max([len(sequence) for sequence in decoderSequences]) # Finds the max length of the decoder sequences
        decoderInputData = pad_sequences(decoderSequences, maxlen=decoderMaxLength, padding='post')  # Pads the data according to the max, making everything the same length ]
            
        return wordIndex, indexWord, encoderInputData, decoderInputData
        

    def MakeInferenceModels(self):

        # Original trained model use
        trainedModel = models.load_model(ChatbotBase.trainingModelPath) # Loads trained model
        encoderInputs = trainedModel.get_layer("Encoder_Input_Layer").input # Gets the encoder inputs 
        encoderEmbeddingLayer = trainedModel.get_layer("Encoder_Embedding_Layer") # Gets the encoder embedding layer definition
        encoderEmbedding = encoderEmbeddingLayer(encoderInputs) # Finds encoder input embeddings
        encoderLSTMLayer = trainedModel.get_layer("Encoder_LSTM_Layer") # Gets the encoder ltm layer
        _, stateH, stateC = encoderLSTMLayer(encoderEmbedding) # Gets the encoder states
        encoderStates = [stateH, stateC] # Stores as list for use
        decoderInputs = trainedModel.get_layer("Decoder_Input_Layer").input # Gets the decoder inputs 
        decoderLSTMLayer = trainedModel.get_layer("Decoder_LSTM_Layer") # Gets the deocoder ltm layer
        decoderEmbeddingLayer = trainedModel.get_layer("Decoder_Embedding_Layer") # Gets the decooder embedding layer definition
        decoderEmbedding = decoderEmbeddingLayer(decoderInputs) # # Finds decoder input embeddings
        decoderDenseLayer = trainedModel.get_layer("Decoder_Dense_Layer") # Gets the decooder dense layer definition


        # Model Definitions
        stateHInputLayer = Input(shape=(ChatbotBase.hiddenLayers,), name = "State_H_Input_Layer") # Input layer for state h, uses hidden layers
        stateCInputLayer = Input(shape=(ChatbotBase.hiddenLayers,), name = "State_C_Input_Layer") # Input layer for state c, uses hidden layers
        
        # Encoder Model
        encoderModel = Model(inputs=encoderInputs, outputs=encoderStates, name = "Inference_Encoder_Model") # Model for encoder of inference model, takes inputs and produces encodser states, uses trained model encoder inputs
        #encoderModel.summary(line_length=220) # Summary of encoder model, commented out for use
        
        #Decoder Inference Model
        decoderStateInputH = stateHInputLayer # Uses state h input layer as decoder input
        decoderStateInputC = stateCInputLayer # Uses state c input layer as decoder input
        decoderStateInputs = [decoderStateInputH, decoderStateInputC] # Creates list of states
        decoderOutputs, stateH, stateC = decoderLSTMLayer(decoderEmbedding, initial_state=decoderStateInputs) #  Uses training model LSTM layer definition with the embeddings to find the outputs of the states input
        decoderStates = [stateH, stateC] # Takes those states as a list for use in next iteration
        decoderOutputs = decoderDenseLayer(decoderOutputs) # Condenses outputs into weights, giving probablity of text 
        decoderModel = Model(inputs = [decoderInputs] + decoderStateInputs, outputs = [decoderOutputs] + decoderStates, name = "Inference_Decoder_Model") # Createes the decoder inferance model using training input and outputs as well as the states
        #decoderModel.summary(line_length=220) # Summarises model, commented out for use

        decoderModel.save(f"{ChatbotBase.inferenceModelsPath}decoderModel.h5") # Saves the model for use in webapp
        encoderModel.save(f"{ChatbotBase.inferenceModelsPath}encoderModel.h5") # Saves the model for use in webapp

In [18]:
chatbotProcess = ChatbotRun() # Otherwise just run the chatbot

Attempting to find existing files
Existing Data Found
Complete
Model: "Inference_Decoder_Model"
____________________________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                            Output Shape                                     Param #                   Connected to                                                             
Decoder_Input_Layer (InputLayer)                                        [(None, None)]                                   0                                                                                                  
____________________________________________________________________________________________________________________________________________________________________________________________________________________________
Decoder_Embedding_La